# Credit Risk — XGBoost (Hosted DB Edition)

In [ ]:
import json, numpy as np, pandas as pd
from pathlib import Path
DATA = Path("data"); MODEL = Path("model")
features = pd.read_csv(DATA/"features_training.csv", index_col=0)
feature_cols = [
    "income_inflow_30d","spend_outflow_30d","pct_gambling_spend_90d","merchant_diversity_90d",
    "avg_debit_amt_30d","num_big_txn_30d","days_since_last_salary","debit_credit_ratio_90d",
    "late_fee_count_90d","debit_txn_count_30d","credit_txn_count_30d","net_cash_flow_30d","cash_withdrawal_90d",
]
X = features[feature_cols].values.astype(float)
y = features["label"].values.astype(int)
X.shape, y.shape, y.mean()


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
clf = XGBClassifier(n_estimators=300, max_depth=4, learning_rate=0.05,
                    subsample=0.9, colsample_bytree=0.9, random_state=42,
                    n_jobs=-1, tree_method="hist")
clf.fit(X, y)
print("In-sample AUC:", round(float(roc_auc_score(y, clf.predict_proba(X)[:,1])), 4))


In [ ]:
from xgboost import Booster
MODEL.mkdir(parents=True, exist_ok=True)
booster = clf.get_booster()
booster.save_model(MODEL/"model.xgb.json")
meta = {"feature_order": [
    "income_inflow_30d","spend_outflow_30d","pct_gambling_spend_90d","merchant_diversity_90d",
    "avg_debit_amt_30d","num_big_txn_30d","days_since_last_salary","debit_credit_ratio_90d",
    "late_fee_count_90d","debit_txn_count_30d","credit_txn_count_30d","net_cash_flow_30d","cash_withdrawal_90d",
], "version": "2.0.0-xgb"}
(MODEL/"feature_order.json").write_text(json.dumps(meta, indent=2))
print("Saved artifacts to", MODEL)


## Connect to hosted PostgreSQL and perform sanity check / scoring (set PGURL)

In [ ]:
import os, pandas as pd, psycopg2
pgurl = os.environ.get("PGURL"); assert pgurl, "Set PGURL per data/DB_CONNECTIONS.md"
conn = psycopg2.connect(pgurl)
print("Connected")


In [ ]:
# Example: compute features for a few scoring customers and predict
from xgboost import Booster, DMatrix
sql = """
WITH expanded AS (
  SELECT
    t.customer_id,
    t.snapshot_date AS S,
    (e->>'ts')::timestamptz AT TIME ZONE 'UTC' AS ts,
    (e->>'amount')::numeric AS amount,
    (e->>'type') AS type,
    (e->>'category') AS category,
    (e->>'merchant') AS merchant,
    (e->>'status') AS status
  FROM bank_transactions_scoring t
  CROSS JOIN LATERAL jsonb_array_elements(t.transactions) AS e
)
SELECT
-- FILL in the query that will extract required features
FROM expanded
GROUP BY customer_id, S
ORDER BY customer_id
LIMIT 50;
"""
feats = pd.read_sql(sql, conn)
meta = json.loads((MODEL/"feature_order.json").read_text())
order = meta["feature_order"]
bst = Booster(); bst.load_model(MODEL/"model.xgb.json")
dm = DMatrix(feats[order].values.astype(float), feature_names=order)
probs = bst.predict(dm)
out = feats[["customer_id","snapshot_date"]].copy()
out["probability"] = probs
out["model_version"] = meta.get("version","unknown")
out.head()


## ✅ Next steps (must-do deliverables)

You now have a trained model in-memory. **You must persist model artefacts and complete deployment tasks.**

### 1) Persist artefacts (required)
- `model/model.xgb.json` — XGBoost booster (saved above)
- `model/feature_order.json` — exact feature order the model expects

> If these files do not exist yet, **re-run the “Save model artefacts” cell above**.

### 2) Prepare for serving (required)
- **Lambda script** (see `starter/lambda_handler.py`): loads the booster, accepts `{"features": {...}}`, returns probability.
- **Docker image** using the provided `starter/Dockerfile` (Lambda base). Build & run locally.
- **Invocation**: Call the container (or lambda-local) with features in the **exact** order from `feature_order.json`.

### 3) Score hosted data (required)
- Connect to the hosted PostgreSQL (see `data/DB_CONNECTIONS.md`).
- Compute features from `bank_transactions_scoring` using the CTE in `feature_spec.md`.
- Respect the feature order from `model/feature_order.json` and produce outputs:
  - Columns: `customer_id`, `snapshot_date`, `probability`, `model_version`
  - Format: CSV and/or JSON

### 4) Submission (required)
- The SQL used for feature computation
- Trained artefacts: `model/model.xgb.json`, `model/feature_order.json`
- Lambda/Docker code and a one-liner to run
- A small README describing how you produced predictions

> Tip: keep data access **read-only**; do not create server-side views/tables in the hosted DB.


In [ ]:
# Sanity check: artefacts exist + hashes for reproducibility
from pathlib import Path
import hashlib, json, os

MODEL = Path("model")
paths = [MODEL/"model.xgb.json", MODEL/"feature_order.json"]
for p in paths:
    assert p.exists(), f"Missing artefact: {p}"

def sha256(path):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

print("OK — artefacts found:")
for p in paths:
    print(f"- {p}  size={os.path.getsize(p)}  sha256={sha256(p)}")

# Show feature order to reduce mismatches at serving time
meta = json.load(open(MODEL/"feature_order.json"))
print("Feature order:", meta["feature_order"])
